# Runnable

### **Overview**

Các đối tượng `Runnable` của LangChain cung cấp một phương pháp mô-đun và linh hoạt để thiết kế quy trình làm việc bằng cách cho phép xâu chuỗi, thực thi song song và biến đổi dữ liệu. Những tiện ích này cho phép xử lý hiệu quả các đầu vào và đầu ra có cấu trúc, với chi phí mã tối thiểu.

**Các thành phần chính là:**

* `RunnableLambda`: Một tiện ích nhẹ cho phép áp dụng logic tùy chỉnh thông qua các hàm lambda, lý tưởng cho việc biến đổi dữ liệu động và nhanh chóng.
* `RunnablePassthrough`: Được thiết kế để truyền dữ liệu đầu vào không thay đổi hoặc tăng cường nó bằng các thuộc tính bổ sung khi kết hợp với phương thức `.assign()`.
* `itemgetter`: Một tiện ích mô-đun `operator` của Python để trích xuất hiệu quả các khóa hoặc chỉ mục cụ thể từ dữ liệu có cấu trúc như từ điển hoặc bộ tuple.

Những công cụ này có thể được kết hợp để xây dựng các quy trình làm việc mạnh mẽ, chẳng hạn như:

* Trích xuất và xử lý các phần tử dữ liệu cụ thể bằng `itemgetter`.
* Thực hiện các biến đổi tùy chỉnh với `RunnableLambda`.
* Tạo các pipeline dữ liệu đầu cuối với chuỗi `Runnable`.

Bằng cách tận dụng các thành phần này, người dùng có thể thiết kế các pipeline có thể mở rộng và tái sử dụng cho các quy trình làm việc học máy và xử lý dữ liệu.


### **RunnablePassthrough là gì?**

`RunnablePassthrough` là một lớp trong LangChain thuộc module `langchain_core.runnables`, được thiết kế để truyền dữ liệu đầu vào qua mà không thay đổi nó hoặc bổ sung thêm các khóa (keys) vào đầu ra nếu cần. Nó hoạt động gần giống như một hàm "identity" (hàm giữ nguyên giá trị đầu vào), nhưng có thể được cấu hình để thêm thông tin bổ sung khi đầu vào là một từ điển (dictionary). 

`RunnablePassthrough` thường được sử dụng cùng với `RunnableParallel` để truyền dữ liệu từ một bước trước đó trong chuỗi (chain) sang một bước sau mà không cần biến đổi, giúp định dạng dữ liệu một cách linh hoạt trong các ứng dụng phức tạp như Retrieval-Augmented Generation (RAG).

### **Khi nào sử dụng RunnablePassthrough?**
- Khi bạn muốn giữ nguyên đầu vào và truyền nó sang bước tiếp theo trong chuỗi.
- Khi cần kết hợp dữ liệu gốc với dữ liệu đã được xử lý trong một chuỗi song song (`RunnableParallel`).
- Đặc biệt hữu ích trong các ứng dụng RAG, nơi bạn cần truyền câu hỏi của người dùng (query) cùng với ngữ cảnh (context) được truy xuất từ bộ nhớ vector.

### **Cách hoạt động của RunnablePassthrough**
- **Đầu vào**: Bất kỳ dữ liệu nào (có thể là chuỗi, số, từ điển, v.v.).
- **Đầu ra**: Dữ liệu đầu vào được giữ nguyên hoặc được bổ sung thêm các khóa nếu sử dụng phương thức `assign`.
- **Tính năng chính**: Là một phần của giao diện `Runnable` tiêu chuẩn, nó hỗ trợ các phương thức như `invoke`, `stream`, `batch`, và cả các phiên bản bất đồng bộ (`ainvoke`, `astream`, `abatch`).

### **Hướng dẫn sử dụng chi tiết**

Để sử dụng `RunnablePassthrough`, bạn cần import nó từ `langchain_core.runnables` và tích hợp nó vào chuỗi xử lý của bạn. Dưới đây là các bước và ví dụ minh họa.

#### **1. Cài đặt môi trường**
Trước tiên, đảm bảo bạn đã cài đặt LangChain:
```bash
pip install langchain
```

#### **2. Ví dụ cơ bản: Truyền dữ liệu không thay đổi**
```python
from langchain_core.runnables import RunnablePassthrough

# Tạo một RunnablePassthrough
passthrough = RunnablePassthrough()

# Thực thi với một đầu vào đơn giản
result = passthrough.invoke("Hello, world!")
print(result)  # Output: "Hello, world!"
```
Trong ví dụ này, `RunnablePassthrough` chỉ đơn giản truyền chuỗi đầu vào mà không thay đổi gì.

#### **3. Kết hợp với RunnableParallel**
`RunnablePassthrough` thường được sử dụng trong `RunnableParallel` để truyền dữ liệu gốc cùng với dữ liệu đã xử lý.

```python
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Tạo một chuỗi song song
runnable = RunnableParallel(
    passed=RunnablePassthrough(),  # Truyền dữ liệu gốc
    modified=lambda x: x + 1       # Xử lý dữ liệu: cộng thêm 1
)

# Thực thi với đầu vào là một từ điển
result = runnable.invoke({"num": 1})
print(result)  # Output: {'passed': {'num': 1}, 'modified': 2}
```
- **Giải thích**: 
  - `passed` giữ nguyên đầu vào `{"num": 1}`.
  - `modified` thực hiện phép tính cộng 1 vào giá trị của `num`, trả về 2.

#### **4. Sử dụng trong RAG (Retrieval-Augmented Generation)**
Dưới đây là một ví dụ thực tế hơn, nơi `RunnablePassthrough` được dùng để truyền câu hỏi của người dùng trong khi truy xuất ngữ cảnh từ một vector store.

```python
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Tạo vector store với dữ liệu mẫu
vectorstore = FAISS.from_texts(
    ["Harrison làm việc tại Kensho"], 
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

# Định nghĩa prompt
template = """Trả lời câu hỏi dựa trên ngữ cảnh sau: {context}
Câu hỏi: {question}"""
prompt = ChatPromptTemplate.from_template(template)

# Khởi tạo mô hình
model = ChatOpenAI()

# Tạo chuỗi truy xuất
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # Truyền câu hỏi gốc
    | prompt
    | model
    | StrOutputParser()
)

# Thực thi chuỗi
result = retrieval_chain.invoke("Harrison làm việc ở đâu?")
print(result)  # Output: "Harrison làm việc tại Kensho"
```
- **Giải thích**:
  - `{"context": retriever, "question": RunnablePassthrough()}`: 
    - `retriever` truy xuất ngữ cảnh từ vector store dựa trên câu hỏi.
    - `RunnablePassthrough()` truyền câu hỏi gốc (`"Harrison làm việc ở đâu?"`) sang bước tiếp theo.
  - Kết quả cuối cùng là câu trả lời dựa trên ngữ cảnh đã truy xuất.

#### **5. Sử dụng với assign để bổ sung dữ liệu**
Phương thức `assign` của `RunnablePassthrough` cho phép thêm các khóa mới vào đầu ra.

```python
from langchain_core.runnables import RunnablePassthrough

# Giả lập một hàm LLM
def fake_llm(prompt: str) -> str:
    return "completion"

# Tạo chuỗi với assign
runnable = {
    'llm1': fake_llm,
    'llm2': fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

# Thực thi
result = runnable.invoke("hello")
print(result)  # Output: {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}
```
- **Giải thích**:
  - Đầu tiên, hai hàm `fake_llm` tạo ra các giá trị `"completion"`.
  - `RunnablePassthrough.assign` giữ nguyên dữ liệu đầu vào và thêm khóa `total_chars` với giá trị là độ dài tổng của `llm1` và `llm2`.

### **Lưu ý quan trọng**
- **Tính linh hoạt**: `RunnablePassthrough` hỗ trợ cả xử lý đồng bộ (`invoke`) và bất đồng bộ (`ainvoke`), phù hợp với các ứng dụng thời gian thực.
- **Kết hợp với LCEL**: Nó là một phần của LangChain Expression Language (LCEL), giúp xây dựng các chuỗi phức tạp một cách dễ dàng với cú pháp pipe (`|`).
- **Hạn chế**: Nếu bạn chỉ cần biến đổi dữ liệu mà không cần giữ nguyên đầu vào, có thể không cần dùng `RunnablePassthrough`.

### **Kết luận**
`RunnablePassthrough` là một công cụ mạnh mẽ và đơn giản trong LangChain để quản lý luồng dữ liệu trong các chuỗi xử lý. Nó đặc biệt hữu ích khi bạn cần truyền dữ liệu gốc qua nhiều bước hoặc kết hợp với dữ liệu đã xử lý. Với các ví dụ trên, bạn có thể bắt đầu tích hợp nó vào dự án của mình một cách hiệu quả.


In [1]:
from dotenv import load_dotenv

load_dotenv(override=True, dotenv_path="../.env")

True

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("What is 10 times {num}")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

chain = prompt | llm | StrOutputParser()

In [3]:
chain.invoke({"num": 5})

'10 times 5 is equal to 50.'

However, with the update to the LangChain library, if the template includes only one variable, it is also possible to pass just the value directly.

In [4]:
chain.invoke(5)

'10 times 5 is equal to 50.'

In [5]:
# Using RunnablePassthrough
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num": 5})

{'num': 5}

In [7]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()


In [8]:
runnable_chain.invoke(4)

'10 times 4 is equal to 40.'

In [9]:
# Input key: num, Assigned key: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

In [14]:
runnable_chain = (RunnablePassthrough.assign(num=lambda x: x["num"] * 3)) | prompt | llm | StrOutputParser()

In [15]:
runnable_chain.invoke({"num": 5})

'10 times 15 is equal to 150.'

## Efficient Parallel Execution with RunnableParallel

`RunnableParallel` là một tiện ích được thiết kế để thực thi nhiều đối tượng `Runnable` đồng thời, hợp lý hóa các quy trình làm việc yêu cầu xử lý song song. Nó phân phối dữ liệu đầu vào trên các thành phần khác nhau, thu thập kết quả của chúng và kết hợp chúng thành một đầu ra thống nhất. Chức năng này biến nó thành một công cụ mạnh mẽ để tối ưu hóa quy trình làm việc nơi các tác vụ có thể được thực hiện độc lập và đồng thời.

**Thực thi đồng thời (Concurrent Execution)**

Thực thi nhiều đối tượng `Runnable` cùng một lúc, giảm thời gian cần thiết cho các tác vụ có thể được song song hóa.

**Quản lý đầu ra thống nhất (Unified Output Management)**

Kết hợp kết quả từ tất cả các lần thực thi song song thành một đầu ra đơn lẻ, mạch lạc, đơn giản hóa quá trình xử lý tiếp theo (downstream processing).

**Tính linh hoạt (Flexibility)**

Có thể xử lý nhiều loại đầu vào đa dạng và hỗ trợ các quy trình làm việc phức tạp bằng cách phân phối khối lượng công việc một cách hiệu quả.


In [16]:
from langchain_core.runnables import RunnableParallel

# Create an instance of RunnableParallel. This instance allows multiple Runnable objects to be executed in parallel.
runnable = RunnableParallel(
    # Pass a RunnablePassthrough instance as the 'passed' keyword argument. This simply passes the input data through without modification.
    passed=RunnablePassthrough(),
    # Use RunnablePassthrough.assign as the 'extra' keyword argument to assign a lambda function 'mult'. 
    # This function multiplies the value associated with the 'num' key in the input dictionary by 3.
    extra=RunnablePassthrough().assign(mult=lambda x: x["num"] * 3),
    # Pass a lambda function as the 'modified' keyword argument. 
    # This function adds 1 to the value associated with the 'num' key in the input dictionary.
    modified=lambda x: x['num'] + 1
)


In [17]:
runnable.invoke({"num": 1})

# {
#     "passed": {"num": 1},
#     "extra": {"num": 1, "mult": 3},
#     "modified": {"num": 2}
# }

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Chains can also be applied to `RunnableParallel`.

In [19]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("Món ăn ngon ở {country} là gì?")
    | llm
    | StrOutputParser()
)

chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("Thủ đô của {country} là gì?")
    | llm
    | StrOutputParser()
)

runnable_chain = RunnableParallel(capital=chain2, food=chain1)
runnable_chain.invoke("Việt Nam")


{'capital': 'Thủ đô của Việt Nam là Hà Nội.',
 'food': 'Việt Nam có rất nhiều món ăn ngon và phong phú, một số món ăn nổi tiếng và được yêu thích bao gồm:\n\n1. Phở: Một trong những món ăn đặc trưng của Việt Nam, phở là một loại súp gồm nước dùng thơm ngon, bún và thịt bò hoặc gà, được ăn kèm với rau sống, giá đỗ và các loại gia vị.\n\n2. Bánh mì: Một loại bánh mì Việt Nam với nhân đa dạng như thịt heo, thịt gà, pate, chả lụa, trứng, rau sống và sốt mayonnaise.\n\n3. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả bao gồm bún (bún đậu hoặc bún rối), thịt nướng và nước mắm chua ngọt.\n\n4. Cơm tấm: Một món ăn phổ biến ở miền Nam Việt Nam, cơm tấm gồm cơm, thịt heo nướng, trứng ốp la, dưa leo, cà chua và nước mắm.\n\n5. Gỏi cuốn: Một món ăn truyền thống của Việt Nam, gỏi cuốn bao gồm các loại rau sống, tôm, thịt, bún và được cuốn trong lá bánh tráng, ăn kèm với nước mắm pha chua ngọt.\n\nĐây chỉ là một số món ăn ngon ở Việt Nam, còn rất nhiều món khác đáng thử khi bạn đến thăm đất n

## Dynamic Processing with RunnableLambda


`RunnableLambda` là một tiện ích linh hoạt cho phép các nhà phát triển xác định logic biến đổi dữ liệu tùy chỉnh bằng cách sử dụng các hàm lambda. Bằng cách cho phép triển khai nhanh chóng và dễ dàng các quy trình xử lý tùy chỉnh, `RunnableLambda` đơn giản hóa việc tạo các pipeline dữ liệu phù hợp trong khi đảm bảo chi phí thiết lập tối thiểu.

**Biến đổi dữ liệu tùy chỉnh (Customizable Data Transformation)**

Cho phép người dùng xác định logic tùy chỉnh để biến đổi dữ liệu đầu vào bằng cách sử dụng các hàm lambda, mang lại tính linh hoạt vô song.

**Nhẹ và đơn giản (Lightweight and Simple)**

Cung cấp một cách đơn giản để triển khai xử lý ad-hoc mà không cần định nghĩa lớp hoặc hàm mở rộng.


In [30]:
from datetime import datetime

def get_today(a=None):
    return datetime.today().strftime("%b-%d")

get_today(None)

'Mar-08'

In [32]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template("List {n} famous people whose birthday is on {today}. Include their date of birth.")

chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
)

chain.invoke(3)

StringPromptValue(text='List 3 famous people whose birthday is on Mar-08. Include their date of birth.')

## Extracting Specific Keys Using `itemgetter`

`itemgetter` là một hàm tiện ích từ module `operator` của Python với các tính năng và lợi ích sau:

**Chức năng cốt lõi**

* Trích xuất hiệu quả các giá trị từ các khóa hoặc chỉ mục cụ thể trong từ điển (dictionaries), bộ (tuples) và danh sách (lists)
* Có khả năng trích xuất nhiều khóa hoặc chỉ mục cùng lúc
* Hỗ trợ phong cách lập trình hàm (functional programming style)

**Tối ưu hóa hiệu suất**

* Hiệu quả hơn so với lập chỉ mục thông thường cho các thao tác truy cập khóa lặp đi lặp lại
* Tối ưu hóa việc sử dụng bộ nhớ
* Lợi thế về hiệu suất khi xử lý các tập dữ liệu lớn (large datasets)

**Sử dụng trong LangChain**

* Lọc dữ liệu trong các thành phần chuỗi (chain compositions)
* Trích xuất có chọn lọc từ các cấu trúc đầu vào phức tạp
* Kết hợp với các đối tượng `Runnable` khác để tiền xử lý dữ liệu (data preprocessing)


In [38]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# Function that returns the length of a sentence.
def length_function(text):
    return len(text)

# Function that returns the product of the lengths of two sentences.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

# Function that uses _multiple_length_function to return the product of the lengths of two sentences.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict['text1'], _dict['text2'])

prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")

chain = (
    {
        "a": itemgetter('word1') | RunnableLambda(len),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")} | RunnableLambda(multiple_length_function)
    }
    | prompt
)


In [39]:
test = {"word1": "Hello", "word2": "World"}
chain.invoke(test)

ChatPromptValue(messages=[HumanMessage(content='What is 5 + 25?', additional_kwargs={}, response_metadata={})])